In [1]:
import os
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion


In [ ]:
from typing import Optional, TypedDict, List, Annotated
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from config import *
kernel = Kernel()

chat_completion = AzureChatCompletion()

class LightModel(TypedDict):
    id: str
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None

class LightsPlugin:
    def __init__(self, lights: List[LightModel]):
        self.lights = lights
    
    @kernel_function
    def get_lights(self) -> List[LightModel]:
        return self.lights
    
    @kernel_function
    def get_stats(self, id: Annotated[int, "The ID of Light"]
                  ) -> Optional[LightModel]:
        """Get the stats of a light by its ID."""
        for light in self.lights:
            if light['id'] == id:
                return light
        return None
    
    @kernel_function
    def change_state(self, id : Annotated[int, "The ID of light"],
                     new_state:LightModel
                     ) -> Optional[LightModel]:
        """Change the state of a light by its ID."""
        for light in self.lights:
            if light['id'] == id:
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return light
        return None
    

In [ ]:
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

plugin = LightsPlugin(lights)
kernel.register_plugin(plugin)

In [ ]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings   

execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

history = ChatHistory()
history.add_user_message("please turn on the lamp")

result = await chat_completion.get_chat_message_content(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
)

print("Assistant response:", str(result))